In [3]:
# pip install zarr-eosdis-store

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.3/865.3 kB 24.1 MB/s eta 0:00:0000:01
  Attempting uninstall: xarray
    Found existing installation: xarray 2023.4.2
    Uninstalling xarray-2023.4.2:
      Successfully uninstalled xarray-2023.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
intake-xarray 0.6.1 requires xarray>=02022, but you have xarray 0.21.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Core libraries for this tutorial
# Available via `pip install zarr zarr-eosdis-store`
from eosdis_store import EosdisStore
import xarray as xr

# Other Python libraries
import requests
from pqdm.threads import pqdm
from matplotlib import animation, pyplot as plt
from IPython.core.display import display, HTML

# Python standard library imports
from pprint import pprint

/tmp/ipykernel_6564/303224034.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [5]:
data_provider = 'POCLOUD'
mur_short_name = 'MUR-JPL-L4-GLOB-v4.1'

In [8]:
plt.rcParams['figure.figsize'] = 12, 6

In [12]:
start_time = '2021-09-01T21:00:00Z'
end_time = '2021-09-30T20:59:59Z'

# lats = slice(35, -5)
# lons = slice(45, 90)

lats = slice(5, 20)
lons = slice(50, 60)

In [13]:
cmr_url = 'https://cmr.earthdata.nasa.gov/search/granules.json'
response = requests.get(cmr_url, 
                        params={
                            'provider': data_provider,
                            'short_name': mur_short_name, 
                            'temporal': f'{start_time},{end_time}',
                            'bounding_box': f'{lons.start},{lats.start},{lons.stop},{lats.stop}',
                            'page_size': 2000,
                            }
                       )
granules = response.json()['feed']['entry']
pprint(granules[0])

{'boxes': ['-90 -180 90 180'],
 'browse_flag': False,
 'collection_concept_id': 'C1996881146-POCLOUD',
 'coordinate_system': 'CARTESIAN',
 'data_center': 'POCLOUD',
 'dataset_id': 'GHRSST Level 4 MUR Global Foundation Sea Surface Temperature '
               'Analysis (v4.1)',
 'day_night_flag': 'UNSPECIFIED',
 'granule_size': '9.059906005859375E-5',
 'id': 'G2113241213-POCLOUD',
 'links': [{'href': 's3://podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20210901090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc',
            'hreflang': 'en-US',
            'rel': 'http://esipfed.org/ns/fedsearch/1.1/s3#',
            'title': 'This link provides direct download access via S3 to the '
                     'granule.'},
           {'href': 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20210901090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc',
            'hreflang': 'en-US',
            'rel': 'http://esipfed.org/ns/fedsearch/1.1/data

In [14]:
urls = []
for granule in granules:
    for link in granule['links']:
        if link['rel'].endswith('/data#'):
            urls.append(link['href'])
            break

In [15]:
response = requests.head(f'{urls[0]}.dmrpp')

print('Can we use EosdisZarrStore and XArray to access these files more efficiently?')
print('Yes' if response.ok else 'No')

Can we use EosdisZarrStore and XArray to access these files more efficiently?
Yes


In [18]:
def open_as_zarr_xarray(url):
    return xr.open_zarr(EosdisStore(url), consolidated=False)

datasets = pqdm(urls, open_as_zarr_xarray, n_jobs=30)

SUBMITTING | :   0%|          | 0/30 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/30 [00:00<?, ?it/s]

/srv/conda/envs/notebook/lib/python3.9/site-packages/zarr/storage.py:2919: UserWarning: Store {'.zgroup': {'zarr_format': 2}, 'mask/.zarray': {'zarr_format': 2, 'filters': [{'id': 'shuffle', 'elementsize': 2}], 'order': 'C', 'dtype': '<i2', 'shape': [1, 17999, 36000], 'compressor': {'id': 'zlib', 'level': 4}, 'chunks': [1, 1447, 2895], 'fill_value': -128}, 'mask/.zattrs': {'long_name': 'sea/land field composite mask', '_FillValue': -128, 'valid_min': 1, 'valid_max': 31, 'flag_masks': [1, 2, 4, 8, 16], 'flag_meanings': 'open_sea land open_lake open_sea_with_ice_in_the_grid open_lake_with_ice_in_the_grid', 'comment': 'mask can be used to further filter the data.', 'coordinates': 'lon lat', 'source': 'GMT \\"grdlandmask\\", ice flag from sea_ice_fraction data', '_ARRAY_DIMENSIONS': ['time', 'lat', 'lon']}, 'mask/.zchunkstore': {'0.0.0': {'offset': 346699658, 'size': 6426}, '0.0.1': {'offset': 346706084, 'size': 4981}, '0.0.2': {'offset': 346711065, 'size': 4083}, '0.0.3': {'offset': 34671

COLLECTING | :   0%|          | 0/30 [00:00<?, ?it/s]

In [19]:
ds = xr.concat(datasets, 'time')
ds

<xarray.Dataset>
Dimensions:           (time: 30, lat: 17999, lon: 36000)
Coordinates:
  * lat               (lat) float32 -89.99 -89.98 -89.97 ... 89.97 89.98 89.99
  * lon               (lon) float32 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * time              (time) datetime64[ns] 2021-09-01T09:00:00 ... 2021-09-3...
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    dt_1km_data       (time, lat, lon) timedelta64[ns] dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    sst_anomaly       (time, lat, lon) float32 dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.7
    title:                      Daily MUR SST, Final product
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    institution:                Jet Propulsion Laboratory
    history:                    created at nominal 4-day latency; replaced nr...
    ...                         ...
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L4
    cdm_data_type:              grid

In [20]:
ds['time'] = ds['time'].dt.floor('D')

In [54]:
import numpy as np
zarr_new = ds.sel(lat=slice(-5, 35), lon=slice(45,90))

zarr_ds = zarr_new.sortby('time')
zarr_ds = zarr_ds.sel(time=slice('2021-09-01', '2021-09-30'))
ds_downsampled = zarr_ds.isel(lat=slice(0, None, 10), lon=slice(0, None, 10))
zarr_ds

<xarray.Dataset>
Dimensions:           (time: 30, lat: 4001, lon: 4501)
Coordinates:
  * lat               (lat) float32 -5.0 -4.99 -4.98 -4.97 ... 34.98 34.99 35.0
  * lon               (lon) float32 45.0 45.01 45.02 45.03 ... 89.98 89.99 90.0
  * time              (time) datetime64[ns] 2021-09-01 2021-09-02 ... 2021-09-30
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(1, 708, 18), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(1, 708, 18), meta=np.ndarray>
    dt_1km_data       (time, lat, lon) timedelta64[ns] dask.array<chunksize=(1, 183, 661), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(1, 183, 661), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(1, 183, 661), meta=np.ndarray>
    sst_anomaly       (time, lat, lon) float32 dask.array<chunksize=(1, 708, 18), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.7
    title:                      Daily MUR SST, Final product
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    institution:                Jet Propulsion Laboratory
    history:                    created at nominal 4-day latency; replaced nr...
    ...                         ...
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L4
    cdm_data_type:              grid

In [63]:
import dask.array as da
from dask.delayed import delayed
from sklearn.model_selection import train_test_split
import gc

def preprocess_day_data(day_data):
    # day_data = da.squeeze(day_data)
    mean_val = da.nanmean(day_data)  # compute here to get scalar value
    return day_data - mean_val

def preprocess_data(zarr_ds, chunk_size=200):
    total_len = zarr_ds['analysed_sst'].shape[0]
    chunk_shape = (chunk_size,) + zarr_ds['analysed_sst'].shape[1:]
    print(chunk_shape)
    chunks = []

    for start_idx in range(0, total_len, chunk_size):
        end_idx = min(start_idx + chunk_size, total_len)
        
        # Directly slice the dask array without wrapping it with da.from_array again
        chunk = zarr_ds['analysed_sst'][start_idx:end_idx]
        
        processed_chunk = chunk.map_blocks(preprocess_day_data)
        
        # Use da.where to replace NaNs with 0.0
        processed_chunk = da.where(da.isnan(processed_chunk), 0.0, processed_chunk)
        chunks.append(processed_chunk)
        
    return da.concatenate(chunks, axis=0)

processed_data = preprocess_data(zarr_ds).compute()

def prepare_data_from_processed(processed_data, window_size=5): 
    length = processed_data.shape[0]
    X, y = [], []

    for i in range(length - window_size):
        X.append(processed_data[i:i+window_size])
        y.append(processed_data[i+window_size])

    X, y = da.array(X), da.array(y)
    return X, y

X, y = prepare_data_from_processed(processed_data)

(200, 4001, 4501)


In [64]:
def time_series_split(X, y, train_ratio=0.7, val_ratio=0.2):
    total_length = X.shape[0]
    
    # Compute end indices for each split
    train_end = int(total_length * train_ratio)
    val_end = int(total_length * (train_ratio + val_ratio))
    
    X_train = X[:train_end]
    y_train = y[:train_end]
    
    X_val = X[train_end:val_end]
    y_val = y[train_end:val_end]
    
    X_test = X[val_end:]
    y_test = y[val_end:]
    
    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = time_series_split(X, y)

In [65]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Conv2D

def create_simple_model(input_shape=(5, 4001, 4501, 1)):
    model = Sequential()

    # ConvLSTM layer
    model.add(ConvLSTM2D(filters=32, kernel_size=(3, 3),
                         input_shape=input_shape,
                         padding='same', return_sequences=False))
    model.add(BatchNormalization())
    
    # Conv2D layer for output
    model.add(Conv2D(filters=1, kernel_size=(3, 3), padding='same', activation='linear'))

    return model

model = create_simple_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_2 (ConvLSTM2D)  (None, 4001, 4501, 32)    38144     
                                                                 
 batch_normalization_2 (Bat  (None, 4001, 4501, 32)    128       
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           (None, 4001, 4501, 1)     289       
                                                                 
Total params: 38561 (150.63 KB)
Trainable params: 38497 (150.38 KB)
Non-trainable params: 64 (256.00 Byte)
_________________________________________________________________


In [ ]:
from tensorflow.keras.layers import Input, Dropout, Dense, Add, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

model.compile(optimizer='adam', loss='mse', metrics=['mse'])

early_stop = EarlyStopping(patience=5, restore_best_weights=True)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.batch(32)

history = model.fit(train_dataset, epochs=20, validation_data=val_dataset, callbacks=[early_stop])

2023-08-08 20:59:20.056070: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6122890340 exceeds 10% of free system memory.
2023-08-08 20:59:26.095593: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1224578068 exceeds 10% of free system memory.
2023-08-08 20:59:28.060444: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1800850100 exceeds 10% of free system memory.
2023-08-08 20:59:29.254748: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6122890340 exceeds 10% of free system memory.
2023-08-08 20:59:31.796230: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1224578068 exceeds 10% of free system memory.


Epoch 1/20


In [53]:
y_train

dask.array<getitem, shape=(17, 4001, 4501), dtype=float32, chunksize=(1, 1023, 2047), chunktype=numpy.ndarray>